document 단위의 classification

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import random
import numpy as np
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, LongformerForSequenceClassification, LongformerTokenizer, DebertaV2ForSequenceClassification
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments, PreTrainedTokenizer
from sklearn.model_selection import train_test_split
from collections import Counter
from transformers import BertModel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from tqdm import tqdm
import torch.nn.functional as F
from types import SimpleNamespace
import re
from torch.optim import AdamW
from sklearn.model_selection import KFold
import os

2025-07-19 20:43:28.510140: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-19 20:43:28.517846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752925408.527233 3321215 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752925408.530034 3321215 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-19 20:43:28.539984: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
CONFIG = {
    "data_base": "../data",
}

In [14]:
train_csv = pd.read_csv(f"{CONFIG['data_base']}/final_aug_train.csv")
test_csv = pd.read_csv(f"{CONFIG['data_base']}/test.csv")
test_csv = test_csv.rename(columns={
    'paragraph_text': 'full_text'
    })

In [4]:
# 기존 데이터에서 클래스별 분리
label_0 = train_csv[train_csv['generated'] == 0]
label_1 = train_csv[train_csv['generated'] == 1]

# 두 클래스 중 작은 수로 균형 맞추기
count = min(len(label_0), len(label_1))

# 무작위 샘플링
sampled_0 = label_0.sample(n=6*count, random_state=42)
sampled_1 = label_1.sample(n=count, random_state=42)

# 균형 잡힌 데이터셋 생성
train_csv = pd.concat([sampled_0, sampled_1]).sample(frac=1, random_state=42).reset_index(drop=True)

# 확인
print(f"✅ 샘플링 완료: 총 {len(train_csv)}개 (각 클래스 {count}개)")
print(train_csv["generated"].value_counts())



train_csv.head()

✅ 샘플링 완료: 총 382508개 (각 클래스 54644개)
generated
0    327864
1     54644
Name: count, dtype: int64


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,paragraph_idx,full_text,generated,document_label
0,1146518.0,1146518.0,1147756.0,오다 노부유키,6.0,"고지(弘治) 2년(1556년) 4월, 노부나가의 지원자였던 장인 사이토 도산(斎藤道...",0,0.0
1,625140.0,625140.0,625879.0,신당역 살인 사건,12.0,서울교통공사는 2022년 9월 15일 오전 7시경 직원들에게 특이사항이 없다고 보고...,0,0.0
2,714042.0,714042.0,714894.0,먼로주의,2.0,영국은 신성동맹의 정책에 반대한다는 뜻을 분명히 했습니다. 중남미 무역이 엄청난 이...,1,1.0
3,194033.0,194033.0,194224.0,닭볶음탕,10.0,"또, 이 요리는 간혹 조리 전에 간단히 볶아 기름을 녹이는 과정을 제외하면 볶음이라...",0,0.0
4,355044.0,355044.0,355457.0,카르멘 (소설),3.0,"주인공은 고고학자로서, 문다 전투의 유적을 답사하기 위해 안내인 안토니오와 같이 스...",0,0.0


In [5]:
# 각 라벨 개수 출력
print(train_csv['generated'].value_counts())

# 전체 개수
total = len(train_csv)

# 각 비율 계산
label_0_ratio = (train_csv['generated'] == 0).sum() / total
label_1_ratio = (train_csv['generated'] == 1).sum() / total

print(f"\n라벨 0 비율: {label_0_ratio:.4f}")
print(f"라벨 1 비율: {label_1_ratio:.4f}")


generated
0    327864
1     54644
Name: count, dtype: int64

라벨 0 비율: 0.8571
라벨 1 비율: 0.1429


In [6]:
train_df, val_df = train_test_split(
    train_csv,
    test_size=0.01,
    random_state=42,
    stratify=train_csv['generated']  # label 분포 유지
)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data_df, tokenizer, mode='train'):
        self.data = data_df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['full_text']

        # 슬라이딩 윈도우 기반 tokenization
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=512,
            stride=256,  # ✅ 겹치게 자름
            return_overflowing_tokens=True,
            return_tensors="pt"
        )

        # segment 중 하나 랜덤 선택
        n_segments = inputs["input_ids"].size(0)
        seg_idx = random.randint(0, n_segments - 1)

        item = {
            k: v[seg_idx] for k, v in inputs.items() if k != "overflow_to_sample_mapping"
        }

        # Longformer는 token_type_ids 없음
        item.pop("token_type_ids", None)

        if self.mode == 'train':
            item["labels"] = int(row["generated"])  # binary classification

        return item


In [8]:
tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

train_dataset = CustomDataset(train_df, tokenizer, mode='train')
val_dataset = CustomDataset(val_df, tokenizer, mode='train')  # 선택적

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = AutoModelForSequenceClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=2  # AI vs HUMAN → 이진 분류
)

base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],  # BERT 계열은 attention 부분 지정
    # target_modules=["query_proj", "value_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(base_model, lora_config)
model

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): ElectraForSequenceClassification(
      (electra): ElectraModel(
        (embeddings): ElectraEmbeddings(
          (word_embeddings): Embedding(35000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): ElectraEncoder(
          (layer): ModuleList(
            (0-11): 12 x ElectraLayer(
              (attention): ElectraAttention(
                (self): ElectraSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
          

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc
    }

In [11]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")  # shape: (B,)
        outputs = model(**inputs)
        logits = outputs.logits  # shape: (B, C)

        # Label Smoothing
        smoothing = 0.0
        num_classes = logits.size(1)
        smoothed_labels = F.one_hot(labels, num_classes=num_classes).float()  # (B, C)
        smoothed_labels = smoothed_labels * (1 - smoothing) + smoothing / num_classes

        probs = torch.softmax(logits, dim=-1)
        ce_loss = - (smoothed_labels * torch.log(probs + 1e-8)).sum(dim=-1)  # shape: (B,)

        # # Focal Loss (gamma=2)
        # gamma = 2.0
        # pt = probs.gather(dim=-1, index=labels.unsqueeze(-1)).squeeze(-1)  # p_t
        # # pt = (probs * smoothed_labels).sum(dim=-1)  # soft p_t
        # focal_weight = (1 - pt) ** gamma  # shape: (B,)

        # 클래스별 weight 적용 (0: 1.0, 1: 12.0)
        class_weights = torch.tensor([1.0, 6.0], device=logits.device)
        example_weights = class_weights[labels]  # shape: (B,)

        loss = (ce_loss * example_weights).mean()

        return (loss, outputs) if return_outputs else loss

In [12]:
training_args = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    #num_train_epochs=1,
    max_steps=300000,
    learning_rate=5e-5,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=10000,
    logging_dir="./",
    logging_steps=10000,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    fp16=True,
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_3321215/2177980789.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
test_dataset = CustomDataset(test_csv, tokenizer, mode='eval')

# 모델 준비
checkpoint_path ="./checkpoint-220000"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# 예측 저장 리스트
all_preds = []
all_probs = []

valid_keys = {"input_ids", "attention_mask"}

# ✅ 4. 추론 루프 (DataLoader 없이)
with torch.no_grad():
    for i in tqdm(range(len(test_dataset)), desc="Running inference without DataLoader"):
        batch = test_dataset[i]  # 단일 샘플 꺼내기
        # 모델이 받는 키만 선택
        inputs = {k: v.unsqueeze(0).to(model.device) for k, v in batch.items() if k in valid_keys}
        inputs.pop("token_type_ids", None)  # 혹시 있을 경우 제거

        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1)

        all_preds.extend(preds.cpu().tolist())
        all_probs.extend(probs.cpu().tolist())

sample_submission = pd.read_csv(f"{CONFIG['data_base']}/sample_submission.csv", encoding='utf-8-sig')
all_AI = [i[1] for i in all_probs]
sample_submission['generated'] = all_AI

sample_submission.to_csv(f"submit.csv", index=False)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/aicontest/anaconda3/envs/construct/lib/python3.11/site-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'qalora_group_size', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_qalora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means t

KeyboardInterrupt: 